In [9]:
# Preparing a model for deployment
# 1. Package an ML model along with its dependencies.
# 2. Containerize the model using Docker.
# 3. Deploy the model locally, and test its performance.
# 4. Optionally, deploy the model to a cloud environment for wider accessibility.

# 1. Package an ML model along with its dependencies.
# Train a simple model and save it
from sklearn.datasets import load_iris # Example dataset
from sklearn.model_selection import train_test_split # For splitting the dataset
from sklearn.linear_model import LogisticRegression # Example model
import joblib

# Load dataset
iris = load_iris() # Example dataset
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42) # Split dataset

# Train model
model = LogisticRegression(max_iter=200) # Initialize model with increased max_iter
model.fit(X_train, y_train) # Train model

# Save model
joblib.dump(model, 'iris_model.joblib') # Save the trained model to a file

['iris_model.joblib']

In [11]:
# Create a Python script for serving the model
# This script will use Flask to serve the model as a RESTful API, allowing it to make predictions when called.
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Load the model
model = joblib.load('iris_model.joblib')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

# Test the Flask app locally
# You can use curl or Postman to send a POST request to the /predict endpoint with a JSON payload containing the features.
# Example curl command:
# curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" -d '{"features": [5.1, 3.5, 1.4, 0.2]}'




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://10.0.0.4:8000
Press CTRL+C to quit


In [14]:
# Containerize the model using Docker.
# Create a Dockerfile to containerize the Flask app

# First, let's create the requirements.txt file
requirements_content = """Flask==2.0.1
scikit-learn==0.24.2
joblib==1.0.1
numpy==1.21.0"""

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

print("requirements.txt created successfully!")

# Create the Dockerfile content
dockerfile_content = """# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8000 available to the world outside this container
EXPOSE 8000

# Define environment variables
ENV FLASK_APP=app.py

# Run app.py when the container launches
CMD ["flask", "run", "--host=0.0.0.0", "--port=8000"]"""

with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("Dockerfile created successfully!")

# To build the Docker image, run this command in the terminal:
# docker build -t iris_model_service .

# To run the Docker container:
# docker run -d -p 8000:8000 iris_model_service

requirements.txt created successfully!
Dockerfile created successfully!


In [21]:
# Push Docker image to Docker Hub (your Docker account)
import subprocess
import os

# Replace 'your_username' with your actual Docker Hub username
docker_username = "tosasilviu"  # Change this to your Docker Hub username
image_name = "iris_model_service"
tag = "latest"

print("Steps to push your Docker image to Docker Hub:")
print("=" * 50)

print("\n1. First, tag your image with your Docker Hub username:")
tag_command = f"docker tag {image_name} {docker_username}/{image_name}:{tag}"
print(f"   {tag_command}")

print("\n2. Login to Docker Hub:")
login_command = "docker login"
print(f"   {login_command}")
print("   (You'll be prompted for your Docker Hub username and password)")

print("\n3. Push the image to Docker Hub:")
push_command = f"docker push {docker_username}/{image_name}:{tag}"
print(f"   {push_command}")

print("\n4. After pushing, you can view your image at:")
print(f"   https://hub.docker.com/r/{docker_username}/{image_name}")

print("\n" + "=" * 50)
print("Executing Docker commands...")

# Step 1: Tag the image
print("\nTagging image...")
result1 = subprocess.run(["docker", "tag", image_name, f"{docker_username}/{image_name}:{tag}"], 
                        capture_output=True, text=True)
if result1.returncode == 0:
    print("✅ Image tagged successfully!")
else:
    print(f"❌ Error tagging image: {result1.stderr}")

# Step 2: Login (you'll need to enter credentials interactively)
print("\nAttempting Docker login...")
print("Note: You may need to enter your Docker Hub credentials manually in the terminal")

# Step 3: Push the image (will only work after successful login)
print(f"\nTo complete the push, run this command in terminal:")
print(f"docker push {docker_username}/{image_name}:{tag}")

print(f"\nAfter pushing, your image will be available at:")
print(f"https://hub.docker.com/r/{docker_username}/{image_name}")

print(f"\nOthers can then pull your image using:")
print(f"docker pull {docker_username}/{image_name}")

Steps to push your Docker image to Docker Hub:

1. First, tag your image with your Docker Hub username:
   docker tag iris_model_service tosasilviu/iris_model_service:latest

2. Login to Docker Hub:
   docker login
   (You'll be prompted for your Docker Hub username and password)

3. Push the image to Docker Hub:
   docker push tosasilviu/iris_model_service:latest

4. After pushing, you can view your image at:
   https://hub.docker.com/r/tosasilviu/iris_model_service

Executing Docker commands...

Tagging image...
✅ Image tagged successfully!

Attempting Docker login...
Note: You may need to enter your Docker Hub credentials manually in the terminal

To complete the push, run this command in terminal:
docker push tosasilviu/iris_model_service:latest

After pushing, your image will be available at:
https://hub.docker.com/r/tosasilviu/iris_model_service

Others can then pull your image using:
docker pull tosasilviu/iris_model_service
